In [ ]:
# 匯入相關所需的模組
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
from pathlib import PurePath
import cv2


from sklearn.model_selection import train_test_split
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D,Input
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical

# 載入數據集
data_path = 'label/'
categories = [i + j for i in 'A23456789TJQK' for j in "shdc"]
suites = 'shdc'
number = 'A23456789TJQK'

number_images = []
number_labels = []
suites_images = []
suites_labels = []


# 讀取圖片和標籤

index = 0
indexToNumber = {}
indexToSuites = {}
for category in categories:
    path = os.path.join(data_path, category)
    suites_class = suites.index(category[1])
    number_class = number.index(category[0])

    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        img_array = cv2.imread(img_path,0)
        img_array = cv2.copyMakeBorder(img_array, 9, 9, 21, 21, cv2.BORDER_REPLICATE)
        img_array = img_array.astype('float32')/255.
        number_images.append(img_array)
        suites_images.append(img_array)
        number_labels.append(number_class)
        suites_labels.append(suites_class)
        indexToNumber[index] = number_class
        indexToSuites[index] = suites_class
        index += 1


# print(img_array)
# print(img_path)
# 將圖片轉換為NumPy數組
number_images = np.array(number_images)
suites_images = np.array(suites_images)
number_labels = np.array(number_labels)
suites_labels = np.array(suites_labels)

one_hot_number_labels = to_categorical(number_labels, 13)
one_hot_suites_labels = to_categorical(suites_labels, 4)


test = 150
plt.imshow(number_images[test], cmap=plt.get_cmap('gray'))
plt.show()

print(number[indexToNumber[test]],suites[indexToSuites[test]])

In [ ]:
import shutil

datagen_train = ImageDataGenerator(
    rotation_range=40,  # 旋转范围
    width_shift_range=0.2,  # 宽度偏移范围
    height_shift_range=0.2,  # 高度偏移范围
    shear_range=0.2,  # 剪切范围
    zoom_range=0.2,  # 缩放范围
    fill_mode='nearest'  # 填充方式
)
# 訓練模型
# history = model.fit(datagen_train.flow(X_train, y_train,batch_size=100), batch_size=1 ,epochs=700,validation_data=(X_test,y_test))

if os.path.exists('./preprocess'):
    shutil.rmtree('./preprocess')

if not os.path.exists('./preprocess'):
    os.makedirs('./preprocess')

i = 0
for batch in datagen_train.flow(number_images.reshape((-1,70,70,1)),y=one_hot_number_labels, batch_size=32, save_to_dir='preprocess', save_prefix='preprocess', save_format='jpeg'):
    i += 1
    if i > 200:
        break  # 生成20张增强后的图像